In [ ]:
from pydantic import BaseModel,Field
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser
import os
import json
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
import re
from typing import List, Dict, Any, Optional
from enum import Enum
from langchain.output_parsers import PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI



class SectionType(str, Enum):
    CONTENT = "content"
    INFO = "info"
    CODE = "code"
    TIP = "tip"

class Section(BaseModel):
    """Model for a lesson section"""
    type: SectionType = Field(description="Type of section")
    title: str = Field(description="Section title")
    content: str = Field(description="Main content")
    language: Optional[str] = Field(default=None, description="Programming language for code sections")
    explanation: Optional[str] = Field(default=None, description="Explanation for code sections")

class DetailedChapter(BaseModel):
    """Model for a detailed chapter with rich content"""
    id: int = Field(description="Chapter ID")
    title: str = Field(description="Chapter title")
    duration: str = Field(description="Duration in readable format (e.g., '15 min')")
    type: str = Field(default="lesson", description="Chapter type")
    sections: List[Section] = Field(description="List of chapter sections")


class Chapter(BaseModel):
    """Model for a course chapter"""
    chapter_number: int = Field(description="Chapter number")
    title: str = Field(description="Chapter title")
    description: str = Field(description="Brief description of chapter content")
    learning_objectives: List[str] = Field(description="Key learning objectives")
    estimated_duration: int = Field(description="Estimated time to complete in Minutes")

class CourseOutline(BaseModel):
    """Model for complete course outline"""
    course_title: str = Field(description="Title of the course")
    course_description: str = Field(description="Brief course description")
    level: str = Field(description="Level of course")
    total_chapters: int = Field(description="Total number of chapters")
    duration: int   = Field(description="Time required to complete the course in Minutes")
    chapters: List[Chapter] = Field(description="List of chapters")
        


os.environ["GROQ_API_KEY"]


llm = ChatGroq(
            temperature=0.3,
            model_name="openai/gpt-oss-20b",
            max_tokens=4000
        )


In [2]:
def generate_course(name,target_audiunce,difficulty,duration):
    
    llm_chain = llm.with_structured_output(CourseOutline)
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """You are an expert course designer and educator. 
    Your task is to create a complete, well-structured course outline.

    Guidelines:
    - The course should have a logical progression.
    - Each chapter should build on the previous one.
    - Learning objectives must be actionable and clear.
    - Estimated durations should align with the total course duration.
    - Keep explanations concise, student-friendly, and practical."""),

        ("human", """Create a course outline for the topic: {course_topic}

    Requirements:
    - Target audience: {target_audience}
    - Course difficulty: {difficulty_level}
    - Total course duration: {course_duration} Months""")
    ])

    chain = prompt_template | llm_chain
    result = chain.invoke({
        "course_topic": name,
        "target_audience":target_audiunce,
        "difficulty_level": difficulty,
        "course_duration": duration
    })

    return result



In [5]:
def generate_chapter_content(chapters):
    idx = 0
    parser = PydanticOutputParser(pydantic_object=DetailedChapter)

    for chapter in chapters:
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", """You are an expert programming instructor. 
        Your task is to expand a course chapter into detailed lesson sections.

        Guidelines:
        - Divide the chapter into multiple sections of type: content, info, code, or tip.
        - CONTENT → main explanation of a concept, written simply for students.
        - INFO → short fact, rule, or important clarification.
        - CODE → code examples (must include "language" and "explanation").
        - TIP → best practices, pitfalls, or learning hacks.
        - Keep sections concise but informative.
        - The final output MUST be valid JSON that matches the following schema:
        {format_instructions}
        """),

            ("human", """Expand the following chapter into structured sections:

        Chapter {chapter_number}: {chapter_title}
        Description: {chapter_description}
        Learning Objectives: {learning_objectives}
        Estimated Duration: {estimated_duration} minutes
        """)
        ]).partial(format_instructions=parser.get_format_instructions())

        
        llm_chain = llm.with_structured_output(DetailedChapter)
        
        chain = prompt_template | llm | parser

        result = chain.invoke({
            "chapter_number": chapter.chapter_number,
            "chapter_title": chapter.title,
            "chapter_description": chapter.description,
            "learning_objectives": chapter.learning_objectives,
            "estimated_duration": chapter.estimated_duration
        })

        idx += 1
        if idx == 1:
            break
    return result


In [4]:
course = generate_course(name="Python",difficulty="Easy",target_audiunce="Begineer",duration=2)
print(course)

course_title='Python for Beginners: A Gentle Introduction' course_description='A two‑month, beginner‑friendly course that takes you from zero knowledge to a solid foundation in Python. Each week blends theory with hands‑on exercises, culminating in small projects that reinforce what you’ve learned.' level='Easy' total_chapters=10 duration=1000 chapters=[Chapter(chapter_number=1, title='Getting Started with Python', description='Install Python, set up a development environment, and run your first script.', learning_objectives=['Install Python and a code editor (VS Code or PyCharm).', 'Run a script from the command line and a REPL.', 'Understand the structure of a Python program.'], estimated_duration=90), Chapter(chapter_number=2, title='Python Basics: Variables, Types, and Operators', description='Learn how to store data, use built‑in types, and perform arithmetic and logical operations.', learning_objectives=['Declare and assign variables of different types.', 'Use arithmetic, compari

In [6]:
chapters = generate_chapter_content(course.chapters)

In [7]:
chapters.dict()

/tmp/ipykernel_12427/971348139.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  chapters.dict()


{'id': 1,
 'title': 'Chapter 1: Getting Started with Python',
 'duration': '90 min',
 'type': 'lesson',
 'sections': [{'type': <SectionType.CONTENT: 'content'>,
   'title': 'Installing Python',
   'content': 'Download the latest Python installer from python.org, run it, and add Python to your system PATH. Verify the installation by running `python --version` in a terminal.',
   'language': None,
   'explanation': None},
  {'type': <SectionType.INFO: 'info'>,
   'title': 'Python 3.x',
   'content': 'Python 2 is end‑of‑life; always install Python 3.x.',
   'language': None,
   'explanation': None},
  {'type': <SectionType.CONTENT: 'content'>,
   'title': 'Setting Up VS Code',
   'content': 'Install Visual Studio Code, then add the official Python extension. Configure the interpreter by selecting the Python executable from the command palette.',
   'language': None,
   'explanation': None},
  {'type': <SectionType.TIP: 'tip'>,
   'title': 'Python Extension Tips',
   'content': 'Enable lin